In [5]:
import pandas as pd
import re
import json
from pathlib import Path
from emoji import replace_emoji
from unidecode import unidecode

In [6]:
def clean_text(text):
    """Loại bỏ emoji và khoảng trắng thừa, giữ lại số, ký hiệu đặc biệt và ngày giờ."""
    # Loại bỏ hoàn toàn emoji
    text = replace_emoji(text, replace='')
    # Loại bỏ khoảng trắng thừa
    text = ' '.join(text.split())
    return text.strip()

In [7]:
def process_excel_file(input_file, output_json='output.json', output_excel='output.xlsx'):
    """Xử lý file Excel để trích xuất và làm sạch dữ liệu bài viết Facebook."""
    try:
        # Đọc file Excel
        df = pd.read_excel(input_file, engine='openpyxl')
        
        # Kiểm tra các cột bắt buộc
        required_columns = ['Page URL', 'Page Name', 'Post URL', 'Post Text']
        if not all(col in df.columns for col in required_columns):
            raise ValueError("File đầu vào phải chứa các cột: Page URL, Page Name, Post URL, Post Text")

        # Khởi tạo biến
        all_posts = []
        post_index = 1
        skipped_posts = 0

        # Xử lý từng dòng
        for _, row in df.iterrows():
            post_text = str(row['Post Text']) if pd.notna(row['Post Text']) else ''
            if not post_text:
                continue

            # Làm sạch post text
            cleaned_text = clean_text(post_text)
            if cleaned_text:
                post = {
                    'index': str(post_index),
                    'page_url': row['Page URL'],
                    'page_name': row['Page Name'],
                    'post_url': row['Post URL'],
                    'post_text': cleaned_text
                }
                all_posts.append(post)
                post_index += 1
            else:
                skipped_posts += 1

        # Ghi vào file JSON
        with open(output_json, 'w', encoding='utf-8') as f:
            json.dump(all_posts, f, ensure_ascii=False, indent=2)

        # Chuyển sang file Excel
        output_df = pd.DataFrame(all_posts)
        output_df.to_excel(output_excel, index=False, engine='openpyxl')

        # In thông báo
        print(f"Đã xử lý thành công {post_index - 1} bài đăng.")
        print(f"Đã bỏ qua {skipped_posts} bài đăng do nội dung trống sau khi làm sạch.")

    except Exception as e:
        print(f"Đã xảy ra lỗi: {str(e)}")

In [8]:
def main():
    """Hàm chính với khả năng tùy chỉnh đường dẫn đầu ra."""
    # Đường dẫn file đầu vào
    input_file = "../../../Facebook Page Posts Scraping/Raw Data/Confessions of HNMU.xlsx"
    
    # Kiểm tra file đầu vào
    if not Path(input_file).is_file() or not input_file.endswith('.xlsx'):
        print("Vui lòng cung cấp đường dẫn file .xlsx hợp lệ.")
        return

    # Thiết lập đường dẫn đầu ra
    output_dir = Path("output")  # Thư mục đầu ra
    output_json = output_dir / "Confessions of HNMU.json" # Đường dẫn file JSON
    output_excel = output_dir / "Confessions of HNMU.xlsx" # Đường dẫn file Excel

    # Tạo thư mục đầu ra nếu chưa tồn tại
    output_dir.mkdir(parents=True, exist_ok=True)

    # In thông tin đường dẫn
    print("\nThông tin đường dẫn:")
    print(f"Thư mục đầu ra: {output_dir}")
    print(f"File JSON: {output_json}")
    print(f"File Excel: {output_excel}")

    # Xử lý dữ liệu
    process_excel_file(input_file, str(output_json), str(output_excel))

if __name__ == "__main__":
    main()


Thông tin đường dẫn:
Thư mục đầu ra: output
File JSON: output\Confessions of HNMU.json
File Excel: output\Confessions of HNMU.xlsx
Đã xử lý thành công 1014 bài đăng.
Đã bỏ qua 0 bài đăng do nội dung trống sau khi làm sạch.
